In [1]:
import os
# from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import Ollama



In [2]:
MODEL = "llama3.1:8b"

In [3]:
model = Ollama(model = MODEL)


C:\Users\Vikramaditya Mishra\AppData\Local\Temp\ipykernel_25064\2283610030.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model = MODEL)


In [4]:
model.invoke("telll me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you smile! Do you want to hear another one?"

In [5]:


loader = PyPDFLoader("pdfs/renewable.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'pdfs/renewable.pdf', 'page': 0}, page_content='What is Renewable Energy?\nRenewable energy uses energy sources\nthat are continually replenished by\nnature—the sun, the wind, water, the\nEarth’s heat, and plants. Renewable\nenergy technologies turn these fuels into\nusable forms of energy—most often elec-\ntricity, but also heat, chemicals, or\nmechanical power.\nWhy Use Renewable Energy?\nToday we primarily use fossil fuels to heat\nand power our homes and fuel our cars.\nIt’s convenient to use coal, oil, and natural\ngas for meeting our energy needs, but we\nhave a limited supply of these fuels on the\nEarth. We’re using them much more\nrapidly than they are being created. Even-\ntually, they will run out. And because of \nsafety concerns and waste disposal prob-\nlems, the United States will retire much of\nits nuclear capacity by 2020. In the mean-\ntime, the nation’s energy needs are\nexpected to grow by 33 percent during the\nnext 20 years. Renewabl

In [6]:
# vectorstore = Chroma.from_documents(
#     documents=pages,
#     collection_name="ollama_embeds",
#     embedding=OllamaEmbeddings(model='nomic-embed-text'),
# )
# retriever = vectorstore.as_retriever()
parser = StrOutputParser()

In [7]:
rag_template = """
Answer the question based only on the following context. 
If you can't answer the question based on the context, reply 
"Sorry. I don't know at this moment":

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(rag_template)
print(prompt.format(context = "Here is some context", question = "Here is a question?"))
# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | rag_prompt
#     | model
#     | StrOutputParser()
# )


Answer the question based only on the following context. 
If you can't answer the question based on the context, reply 
"Sorry. I don't know at this moment":

Context: Here is some context

Question: Here is a question?



In [8]:
chain = prompt | model | parser

In [9]:
chain.invoke({
    "context": "I am enjoying Jaipur places.",
    "question" : " Where am I right now?"
})

'Jaipur.'

In [10]:
embeddings = OllamaEmbeddings(model=MODEL)



C:\Users\Vikramaditya Mishra\AppData\Local\Temp\ipykernel_25064\3734046490.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


In [11]:
# vectorstore = Chroma.from_documents(
#     documents=pages,
#     collection_name="ollama_embeds",
#     embedding=OllamaEmbeddings(model=MODEL),
# )
# retriever = vectorstore.as_retriever()
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [12]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(pages))]

In [13]:
vector_store.add_documents(documents=pages, ids=uuids)
vector_store.as_retriever().invoke("energy")

[Document(id='10ce38e5-52d0-4852-8246-a013ae161ac5', metadata={'source': 'pdfs/renewable.pdf', 'page': 3}, page_content='geothermal energy. Direct-use applications\nrequire geothermal temperatures between\nabout 70° to 302°F—lower than those\nrequired for electricity generation. The\nUnited States already has about 1,300 geot-\nhermal direct-use systems in operation. \nIn a direct-use system, a well is drilled\ninto a geothermal reservoir, which pro-\nvides a steady stream of hot water. Some\nsystems use the water directly, but most\npump the water through what’s called a\nheat exchanger.The heat exchanger keeps\nthe water separate from a working fluid\n(usually water or a mixture of water and\nantifreeze), which is heated by the geot-\nhermal water. The working fluid then\nflows through piping, distributing the\nheat directly for its intended use. \nThe heated water or fluid can be used in a\nbuilding to replace the traditional heat\nsource—often natural gas—of a boiler, fur-\nnace, a

In [14]:
retriever = vector_store.as_retriever()

In [15]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [16]:
questions = [
    "What is renewable energy?",
    "What are types of renewbale energy?",
    "Explain Wind Energy?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is renewable energy?
Answer: Renewable energy refers to sources of energy that can be replenished naturally over time and are not depleted when used. Examples include:

* Solar energy
* Wind energy
* Geothermal energy (heat from the Earth)
* Hydro energy (energy from water, such as rivers or ocean tides)
* Biomass energy (energy from organic matter, such as plants or waste)

These sources of energy are often contrasted with non-renewable energy sources, which can be depleted when used and take a long time to form again. Examples of non-renewable energy sources include fossil fuels (coal, oil, and natural gas).

Question: What are types of renewbale energy?
Answer: According to the text, there are several types of renewable energy mentioned:

1. **Solar Energy**: This includes photovoltaic systems and dish/Stirling solar power systems.
2. **Wind Energy**: Captured using wind turbines that use new blade designs or airfoils to convert wind energy into electricity.
3. **Geot

In [25]:
import os
import gradio as gr
from datetime import datetime
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from uuid import uuid4
import pandas as pd

# Initialize model and embeddings
MODEL = "llama3.1:8b"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

# Metadata-enabled vector store
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
metadata_store = {}  # Store metadata for documents (keyed by UUID)

# Feedback store as a Pandas DataFrame
feedback_store = pd.DataFrame(columns=["Question", "Correct Answer", "Feedback"])

# Prompt template
rag_template = """
Answer the question based only on the following context. 
If you can't answer the question based on the context, reply 
"Sorry. I don't know at this moment":

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(rag_template)
parser = StrOutputParser()

# Functions
def upload_and_process_pdf(file, tags, category):
    loader = PyPDFLoader(file.name)
    pages = loader.load_and_split()
    
    uuids = [str(uuid4()) for _ in range(len(pages))]
    vector_store.add_documents(documents=pages, ids=uuids)
    
    # Add metadata
    timestamp = datetime.now().isoformat()
    for uuid in uuids:
        metadata_store[uuid] = {"tags": tags, "category": category, "timestamp": timestamp}
    
    return f"PDF processed successfully with {len(pages)} pages!"

def ask_question(question, filter_tags=None, filter_category=None):
    retriever = vector_store.as_retriever()
    filtered_contexts = []
    
    # Apply metadata filters
    for uuid, metadata in metadata_store.items():
        if filter_tags and not any(tag in metadata["tags"] for tag in filter_tags.split(",")):
            continue
        if filter_category and metadata["category"] != filter_category:
            continue
        
        # Invoke retriever and ensure it appends text
        retrieved = retriever.invoke(question)
        if isinstance(retrieved, list):
            # Extract content from Document objects
            filtered_contexts.extend([doc.page_content for doc in retrieved if hasattr(doc, "page_content")])
        elif isinstance(retrieved, str):
            filtered_contexts.append(retrieved)
    
    if not filtered_contexts:
        return "No matching context found for the specified filters."
    
    context = "\n\n".join(filtered_contexts)
    chain = (
        {
            "context": lambda x: context,
            "question": lambda x: question,
        }
        | prompt
        | model
        | parser
    )
    return chain.invoke({"question": question})


def submit_feedback(question, correct_answer, user_feedback):
    global feedback_store
    new_feedback = pd.DataFrame(
        [{"Question": question, "Correct Answer": correct_answer, "Feedback": user_feedback}]
    )
    feedback_store = pd.concat([feedback_store, new_feedback], ignore_index=True)
    return "Feedback submitted successfully!"


def view_feedback():
    global feedback_store
    return feedback_store

def delete_feedback(index):
    global feedback_store
    if index >= 0 and index < len(feedback_store):
        feedback_store = feedback_store.drop(index).reset_index(drop=True)
        return "Feedback entry deleted successfully!"
    else:
        return "Invalid index. Please try again."

def retrain_embeddings():
    global feedback_store, vector_store
    retrained_data = feedback_store[["Correct Answer"]].dropna()
    retrained_uuids = [str(uuid4()) for _ in range(len(retrained_data))]
    retrained_docs = [{"text": ans} for ans in retrained_data["Correct Answer"]]
    vector_store.add_documents(documents=retrained_docs, ids=retrained_uuids)
    return "Retraining completed successfully!"

# Gradio app
with gr.Blocks() as qa_demo:
    gr.Markdown("### Enhanced PDF-Based Question Answering System with Knowledge Management and Feedback")
    
    with gr.Row():
        with gr.Column():
            pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
            tags_input = gr.Textbox(label="Tags (comma-separated)")
            category_input = gr.Textbox(label="Category")
            process_button = gr.Button("Process PDF")
            status_output = gr.Textbox(label="Status", interactive=False)
        
        with gr.Column():
            question_input = gr.Textbox(label="Ask a Question")
            filter_tags_input = gr.Textbox(label="Filter by Tags (optional)")
            filter_category_input = gr.Textbox(label="Filter by Category (optional)")
            answer_button = gr.Button("Get Answer")
            answer_output = gr.Textbox(label="Answer", interactive=False)
    
    with gr.Row():
        gr.Markdown("### Feedback Section")
        feedback_question_input = gr.Textbox(label="Question")
        correct_answer_input = gr.Textbox(label="Correct Answer")
        user_feedback_input = gr.Textbox(label="Your Feedback")
        feedback_button = gr.Button("Submit Feedback")
        feedback_status_output = gr.Textbox(label="Feedback Status", interactive=False)
    
    with gr.Row():
        gr.Markdown("### Feedback Management")
        feedback_table = gr.Dataframe(label="Feedback Entries")
        update_feedback_button = gr.Button("Reload Feedback Table")
        feedback_index_input = gr.Number(label="Delete Feedback at Index", precision=0)
        delete_feedback_button = gr.Button("Delete Feedback")
        delete_feedback_status_output = gr.Textbox(label="Delete Status", interactive=False)
        retrain_button = gr.Button("Retrain Embeddings")
        retrain_status_output = gr.Textbox(label="Retrain Status", interactive=False)
    
    # Button actions
    process_button.click(
        upload_and_process_pdf,
        inputs=[pdf_input, tags_input, category_input],
        outputs=status_output,
    )
    answer_button.click(
        ask_question,
        inputs=[question_input, filter_tags_input, filter_category_input],
        outputs=answer_output,
    )
    feedback_button.click(
        submit_feedback,
        inputs=[feedback_question_input, correct_answer_input, user_feedback_input],
        outputs=feedback_status_output,
    )
    update_feedback_button.click(view_feedback, outputs=feedback_table)
    delete_feedback_button.click(delete_feedback, inputs=feedback_index_input, outputs=delete_feedback_status_output)
    retrain_button.click(retrain_embeddings, outputs=retrain_status_output)

qa_demo.launch()


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\anaconda\envs\knowledge\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\anaconda\envs\knowledge\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\anaconda\envs\knowledge\lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "c:\anaconda\envs\knowledge\lib\site-packages\gradio\blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\anaconda\envs\knowledge\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\anaconda\envs\knowledge\lib\site-packages\anyio\_backends\_asyncio.py", line 2364, in run_sync_in_worker_thread
    return await future
  File "c:\anaconda\envs\knowledge\lib\site-pack